In [7]:
# 初回のみ
%cd /app/notebook/
!git clone https://github.com/Vision-CAIR/MiniGPT-4.git

/app/notebook
Cloning into 'MiniGPT-4'...
remote: Enumerating objects: 289, done.
remote: Counting objects: 100% (220/220), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 289 (delta 119), reused 105 (delta 105), pack-reused 69
Receiving objects: 100% (289/289), 45.35 MiB | 9.19 MiB/s, done.
Resolving deltas: 100% (127/127), done.


In [8]:
# 初回のみ
%cd /app/notebook/MiniGPT-4
!git checkout 22d8888 # latest version as of July 31, 2023.
!wget https://huggingface.co/rinna/bilingual-gpt-neox-4b-minigpt4/resolve/main/customized_mini_gpt4.py
!wget https://huggingface.co/rinna/bilingual-gpt-neox-4b-minigpt4/resolve/main/checkpoint.pth

/app/notebook/MiniGPT-4
Note: switching to '22d8888'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 22d8888 Update README.md
--2023-08-01 02:07:40--  https://huggingface.co/rinna/bilingual-gpt-neox-4b-minigpt4/resolve/main/customized_mini_gpt4.py
Resolving huggingface.co (huggingface.co)... 18.172.52.106, 18.172.52.38, 18.172.52.61, ...
Connecting to huggingface.co (huggingface.co)|18.172.52.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5964 (5.8K) [text/plain]
Saving to: ‘cust

In [6]:
# 次回以降はここから
%cd /app/notebook/MiniGPT-4


[Errno 2] No such file or directory: '/app/notebook/MiniGPT-4'
/app/notebook


In [9]:
import torch
import requests
from PIL import Image
from minigpt4.processors.blip_processors import Blip2ImageEvalProcessor
from customized_mini_gpt4 import CustomizedMiniGPT4

ckpt_path = "./checkpoint.pth"

model = CustomizedMiniGPT4(gpt_neox_model="rinna/bilingual-gpt-neox-4b")
tokenizer = model.gpt_neox_tokenizer

if torch.cuda.is_available():
    model = model.to("cuda")

if ckpt_path is not None:
    print("Load BLIP2-LLM Checkpoint: {}".format(ckpt_path))
    ckpt = torch.load(ckpt_path, map_location="cpu")
    model.load_state_dict(ckpt['model'], strict=False)

vis_processor = Blip2ImageEvalProcessor()

Loading VIT
freeze vision encoder
Loading VIT Done
Loading Q-Former
freeze Qformer
Loading Q-Former Done
Loading LLM


You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


Loading LLM Done
Load BLIP2-LLM Checkpoint: ./checkpoint.pth


In [10]:
def inference(image_url):
    raw_image = Image.open(requests.get(image_url, stream=True).raw).convert('RGB')
    image = vis_processor(raw_image).unsqueeze(0).to(model.device)
    image_emb = model.encode_img(image)
    prompt = [
        {
            "speaker": "ユーザー",
            "text": "<Img><ImageHere></Img> 何が見えますか？"
        },
    ]
    prompt = [
        f"{uttr['speaker']}: {uttr['text']}"
        for uttr in prompt
    ]
    prompt = "\n".join(prompt)
    prompt = (
        prompt
        + "\n"
        + "システム: "
    )
    print(prompt)

    embs = model.get_context_emb(prompt, [image_emb])

    output_ids = model.gpt_neox_model.generate(
        inputs_embeds=embs,
        max_new_tokens=512,
        do_sample=True,
        temperature=1.0,
        top_p=0.85,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
    print(output)

In [14]:
inference("https://ogre.natalie.mu/media/news/comic/2023/0325/jyuyutsu2_PV1_06.jpg?impolicy=hq&imwidth=730&imdensity=1")

ユーザー: <Img><ImageHere></Img> 何が見えますか？
システム: 
絵


In [16]:
inference("https://upload.wikimedia.org/wikipedia/commons/1/15/Red_Apple.jpg")

ユーザー: <Img><ImageHere></Img> 何が見えますか？
システム: 
りんごです。


In [94]:
inference("https://img.cpcdn.com/recipes/4174680/640x480c/13c9ffb6dfb8c32eacda7b8b4628208c.jpg?u=11688791&p=1478826339")

リンゴの断面


In [100]:
inference("https://collabo-cafe.com/wp-content/uploads/cadbb15fbe6e5c8c2878027820dfde30.jpg")

アニメキャラ、キャラクター、キャラ絵、アニメ、キャラ、漫画、キャラ絵


In [106]:
inference("https://static.mercdn.net/c!/w=240/thumb/photos/m55026016864_1.jpg?1689509667")

アニメの女の子


In [111]:
inference("https://blog.ikehikoshop.jp/wp-content/uploads/2021/09/coin.jpg")

洗濯機です。


In [125]:
inference("https://ogre.natalie.mu/media/news/comic/2022/1217/5_toji.jpg?imwidth=750&imdensity=1")

黒い布が巻かれた手
